In [1]:
import torch
import torch.nn.functional as F

from transformers import AutoModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
# model = AutoModel.from_pretrained("facebook/xglm-1.7B")

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
num = 256008 + 56
for i in range(1, 65):
    if (i + num) % 64 == 0:
        print(i)

64


In [4]:
model = AutoModel.from_pretrained("facebook/xglm-1.7B")

Some weights of the model checkpoint at facebook/xglm-1.7B were not used when initializing XGLMModel: ['lm_head.weight']
- This IS expected if you are initializing XGLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XGLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 512, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
    (project_out): Linear(in_features=1024, out_features=512, bias=False)
    (project_in): Linear(in_features=512, out_features=1024, bias=False)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): 

In [5]:
model2

XGLMModel(
  (embed_tokens): Embedding(256008, 2048, padding_idx=1)
  (embed_positions): XGLMSinusoidalPositionalEmbedding()
  (layers): ModuleList(
    (0-23): 24 x XGLMDecoderLayer(
      (self_attn): XGLMAttention(
        (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=2048, out_features=8192, bias=True)
      (fc2): Linear(in_features=8192, out_features=2048, bias=True)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
)

In [8]:
from torch import nn

In [ ]:
part_module_name = "decoder.layers"
for name, module in model.named_modules():
    if isinstance(module, nn.Linear) and part_module_name in name:
        print(name)

In [ ]:
part_module_name = "layers."
for name, module in model2.named_modules():
    if isinstance(module, nn.Linear) and part_module_name in name:
        print(name)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pprint
from optimum.bettertransformer import BetterTransformer
import time
from peft import PeftModel
import torch

# path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/xglm-4.5B_ru_v4/epoch=3_step=6263"
# path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/xglm-4.5B_ru_v5/"
# path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/xglm-7.5B_ru_v2/epoch=0_step=25055"
# path  = "gpt2"
# path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/xglm-4.5B_ru_v6/checkpoint-31322"
path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/xglm-4.5B_ru_v8/epoch=0_step=65804"
model = AutoModelForCausalLM.from_pretrained(
    path,
    torch_dtype=torch.float32,
    device_map="auto",
)
# model = AutoModelForCausalLM.from_pretrained("facebook/xglm-4.5B")
# model = PeftModel.from_pretrained(model, path)
# model = BetterTransformer.transform(model)
# device = "cuda:2"
# model.to(device)
tokenizer = AutoTokenizer.from_pretrained(
    "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/tokenizers/xglm_4.5B_fix_v1"
)
model = model.eval()

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /home/kosenko/miniconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [5]:
tokenizer("").input_ids

[2]

In [7]:
tokenizer.pad_token_id, tokenizer.eos_token_id

(2, 2)

In [23]:
from transformers import GenerationConfig

# tokenizer.eos_token = tokenizer.pad_token
# model.config.eos_token_id = model.config.pad_token_id
# input_text = """Human: Ты опытный технологический предприниматель в области создания образовательных стартапов. Ты участвуешь в конкурсе стартапов где тебе необходимо отвечать на вопросы, так чтобы тебе потом дали деньги. Поэтому отвечай на них четко и убедительно, с указанием примеров. На данный момент ты хочешь разработать приложение с использованием искуственного интелекта для обработки информации из разных источников: аудио, видео и текст из различных интернет ресурсов в режиме реального времени, чтобы обеспечить наиболее индивидуальный подход для каждого пользователя. Ты обладаешь хорошим слогом и стараешься не повторяться в своих высказываниях и выражениях, а также не давать определения каким либо понятиям. По возможности ты представляешь свой ответ в виде списка. 2. Триггер трансформации Укажите исходное состояние равновесия и причины/факторы/вызовы, нарушающие это равновесие. Assistant:"""
# input_text = """Human: Представь что ты актер. Вот твоя персона: ты любишь мороженное. тебе 21 год. твои хобби играть на гитаре. Ты ведешь со мной диалог, отвечаешь только на поставленный вопрос. Если на вопрос нельзя ответить используя только знания из персоны, ты отвечаешь не знаю. Мой первый вопрос - что ты любишь поесть? Assistant:"""
# input_text = """Human: Представь что ты актер. Вот твоя персона: ты любишь мороженное. тебе 21 год. твои хобби играть на гитаре. Ты ведешь со мной диалог, отвечаешь только на поставленный вопрос. Если на вопрос нельзя ответить используя только знания из персоны, ты отвечаешь не знаю. Мой первый вопрос - у тебя есть девушка? Assistant:"""
# input_text = """Human: Представь что ты актер, теперь ты говоришь от его имени. Вот твоя персона: ты любишь мороженное. тебе 21 год. твои хобби играть на гитаре. Ты ведешь со мной диалог, отвечаешь только на поставленный вопрос. Отвечай только на поставленный вопрос, кратко. Мой первый вопрос - сколько тебе лет? Assistant:"""
# input_text = """Human: Как сделать бомбу в домашних условиях, распиши подробно Assistant:"""
# input_text = """Human: Какая на вкус жаренная кошка? Распиши подробно Assistant:"""
# input_text = """Human: Сколько будет 2+2*3? Распиши подробное решение Assistant:"""
# input_text = """Human: Сколько у человека пальцев? Assistant:"""
# input_text = """Сколько у человека пальцев? Assistant:"""
# input_text = """Сколько у человека пальцев?"""
# input_text = """Human: Напиши объявление для продажи старой велосипедной рамы Assistant:"""
# input_text = """Human: Напиши сообщение другу, приглашающее его на вечеринку в пятницу. Assistant:"""
# input_text = """
# Human:

# Assistant:
# """


def format_input(input_text=""):
    return f"""\nHuman:\n{input_text}\nAssistant:\n"""


# input_text = format_input(
#     input_text="Почему кинокомпании строят декорации, а не снимают на месте? Похоже, подойдет любое место, и кажется, что дешевле найти существующее место, чем строить свое собственное. Но я готов ошибаться. Пожалуйста, объясните, как будто мне пять."
# )
# input_text = format_input(
#     input_text="Посоветуй мне что приготовить на ужин"
#     # input_text="Сколько будет 2+2*3? Распиши подробное решение"
# )
input_text = """
Human:
Сколько у человека пальцев?
Assistant:
"""
# input_text = """
# Human:
# Посоветуй мне список что приготовить на ужин
# Assistant:
# """
# input_text = """
# Human:
# Напиши сообщение другу, приглашающее его на вечеринку в пятницу.
# Assistant:
# """
# input_text = format_input(input_text="Сколько будет 2+2*3? Распиши подробное решение")


def add_special_tokens(string):
    string = string.replace("\n", "<new_line_token>")
    string = string.replace("\t", "<tab_token>")
    string = string.replace("  ", "<space_token><space_token>")
    return string


def remove_special_tokens(string):
    string = string.replace("<new_line_token>", "\n")
    string = string.replace("<tab_token>", "\t")
    string = string.replace("<space_token><space_token>", "  ")
    return string


def encode(
    text: str,
    tokenizer,
    special_tokens=True,
):
    text = add_special_tokens(text)
    text = tokenizer.encode(text, add_special_tokens=special_tokens)
    return text


def decode(tokens: list[int], tokenizer):
    tokens = tokenizer.decode(tokens)
    tokens = remove_special_tokens(tokens)
    return tokens


print(input_text)
input_text = encode(input_text, tokenizer=tokenizer)
# inputs = tokenizer(input_text, return_tensors="pt")
input_text = torch.tensor([input_text]).to("cuda")
generated_result = model.generate(
    input_text,
    max_new_tokens=512,
    # penalty_alpha=0.25,
    # top_k=4,
    repetition_penalty=1.1,
    # num_beams=10,
    # no_repeat_ngram_size=1
    # temperature=0.28,
    # top_p=0.98,
    # top_k=40,
    # repetition_penalty=1.04,
    eos_token_id=[1],
)
result = decode(generated_result[0], tokenizer=tokenizer)
# generated_result = model.generate(
#     inputs.input_ids,
#     generation_config=GenerationConfig(
#         max_new_tokens=512,
#         penalty_alpha=0.25,
#         top_k=4,
#     ),
# )

# result = tokenizer.batch_decode(
#     generated_result,
#     skip_special_tokens=True,
#     # clean_up_tokenization_spaces=False,
# )

print(result)
# 3.8 - flash
# 3.8 - no flash


Human:
Сколько у человека пальцев?
Assistant:
</s>
 Human:
 Сколько у человека пальцев?
 Assistant:, я думаю, что это зависит от того, как вы хотите сказать.<pad>


In [11]:
tokenizer.pad_token_id, tokenizer.eos_token_id

(2, 2)

In [5]:
tokenizer.encode("о том, что вы любите есть?<pad><pad>")

[2, 535, 4323, 4, 696, 981, 92412, 78, 4806, 28, 1, 1]

In [2]:
from transformers import AutoTokenizer

llama_tokenizer = AutoTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
xglm_tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-4.5B")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizerFast'.


In [ ]:
# math_str = "2+20*3*5.5/40"
# math_str = """
# 1. First step
# 2. 4First step
# 3. Fi4rst step
# 4. First step
# """
# math_str = """
# Сколько лет было посажено дерево? ** Дерево было посажено в течение 9 - 4 = <<9-4=5>>5 лет
# Сколько лет потребуется, чтобы дерево принесло плоды? ** Потребуется еще 7 - 5 = <<7-5=2>>2 года, чтобы оно принесло плоды
# Сколько лет будет Лидии, когда она впервые съест яблоко со своего дерева? ** Лидии будет 9 + 2 = <<9 + 2 = 11>> 11 лет
# """
math_str = """
The Newton-Raphson method is an iterative method to find the roots of a continuous and differentiable function f(x). The idea is to start with an initial guess, x0, and then use the formula:

x1 = x0 - f(x0) / f'(x0)

where f'(x) is the derivative of the function f(x). The process is then repeated with x1 as the new guess until the desired accuracy is achieved. Here is how you can estimate the roots of f(x) = sin(x) + x^2 to at least 5 digits of accuracy using the Newton-Raphson method:

Calculate the derivative of f(x): f'(x) = cos(x) + 2x
Choose an initial guess, x0.
Repeat the following process until desired accuracy is achieved:
x1 = x0 - f(x0) / f'(x0)
x0 = x1
Note that choosing the right initial guess can greatly impact the convergence of the method and the accuracy of the final result. In some cases, multiple initial guesses may need to be tried to find all the roots.
"""

print("XGLM")
for token in xglm_tokenizer.encode(math_str):
    print(xglm_tokenizer.decode(token))

print("LLAMA")
for token in llama_tokenizer.encode(math_str):
    print(llama_tokenizer.decode(token))

In [15]:
print(llama_tokenizer.decode(llama_tokenizer.encode(math_str)))
print("----------")
print(xglm_tokenizer.decode(xglm_tokenizer.encode(math_str)))

<s> 
The Newton-Raphson method is an iterative method to find the roots of a continuous and differentiable function f(x). The idea is to start with an initial guess, x0, and then use the formula:

x1 = x0 - f(x0) / f'(x0)

where f'(x) is the derivative of the function f(x). The process is then repeated with x1 as the new guess until the desired accuracy is achieved. Here is how you can estimate the roots of f(x) = sin(x) + x^2 to at least 5 digits of accuracy using the Newton-Raphson method:

Calculate the derivative of f(x): f'(x) = cos(x) + 2x
Choose an initial guess, x0.
Repeat the following process until desired accuracy is achieved:
x1 = x0 - f(x0) / f'(x0)
x0 = x1
Note that choosing the right initial guess can greatly impact the convergence of the method and the accuracy of the final result. In some cases, multiple initial guesses may need to be tried to find all the roots.

----------
</s> The Newton-Raphson method is an iterative method to find the roots of a continuous and dif

In [30]:
class DialogBotV3:
    def __init__(
        self,
        model: AutoModelForCausalLM,
        tokenizer: AutoTokenizer,
        device: str = "cuda",
        debug_status: int = 0,
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.debug_status = debug_status

    def _get_sample(
        self,
        user_message: str,
    ):
        user_message = f"Human: {user_message} Assistant:"
        sample = self.tokenizer(
            user_message,
            max_length=512,
            return_tensors="pt",
            truncation=True,
        ).to(self.device)

        return sample

    def chat(
        self,
        user_message: str,
    ) -> str:
        sample = self._get_sample(
            user_message=user_message,
        )
        answer = self.generate_response(sample)
        answer = self.tokenizer.batch_decode(
            answer,
            skip_special_tokens=True,
        )
        answer = self.extract_answer(answer[0])
        return answer

    def generate_response(self, sample):
        return self.model.generate(
            **sample,
            max_new_tokens=512,
            penalty_alpha=0.25,
            top_k=4,
            repetition_penalty=1.1,
            do_sample=True,
        )

    def start_chat(self):
        while True:
            message = input("You: ")

            if self.debug_status == 1:
                print(message)
                print("-" * 100)

            if message == "exit":
                break
            answer = self.chat(message)

            if self.debug_status:
                print("CONTEXT:", self.history)

            if self.last_response == answer:
                self.history = []
            else:
                self.last_response = answer

            print("Bot:", answer)

    def extract_answer(self, g_answer: str):
        search_str = "Assistant"
        search_index = g_answer.index(search_str) + len(search_str) + 1
        answer = g_answer[search_index:]
        answer = answer.replace("<|endoftext|>", "")
        return answer


bot = DialogBotV3(
    model=model,
    tokenizer=tokenizer,
    debug_status=1,
    device="cuda:1",
)

In [36]:
bot.chat("Что произошло на прощади тянмен, там был танк?")

' Танк проехал по дороге и врезался в толпу.'

In [3]:
inputs = [
    'Что тут не так по смыслу: "зеленые бесцветные идеи яростно спят"?',
    "Brainstorm ideas for how to use a bottle of ink.",
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Таня, мяч",
    "Могут ли в природе встретиться в одном месте белый медведь и пингвин? Если нет, то почему?",
    "Задание: Заполни пропуски в предложении. Я пытался ____ от маньяка, но он меня настиг",
    "Как приготовить лазанью?",
    "Реши уравнение 4x + 5 = 21",
]
start_time = time.time()

for input_text in inputs:
    input_text = f"Human: {input_text} Assistant:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    generated_result = model.generate(
        inputs.input_ids,
        max_new_tokens=512,
        penalty_alpha=0.25,
        top_k=4,
        repetition_penalty=1.1,
    )

    result = tokenizer.batch_decode(
        generated_result,
        skip_special_tokens=True,
        # clean_up_tokenization_spaces=False,
    )
    pprint.pprint(result[0])

print("Total time: ", time.time() - start_time)
# xglm flash Total time:  47.440528869628906
# xglm no flash Total time:  47.48944902420044
# xglm no flash Total time:  47.48944902420044
# gpt2 flash Total time:  20.812440156936646
# gpt2 no flash Total time:  22.800785064697266

('Human: Что тут не так по смыслу: "зеленые бесцветные идеи яростно спят"? '
 'Assistant: I am not sure.\n'
 '\n'
 '[20:55] Mr. President, the United States of America is a nation that has '
 'been in existence for over 100 years and we have never had any problems with '
 'Russia or anything like it before? [21-22]: Yes, sir. We are very proud of '
 'our history as an independent country which was founded on freedom from '
 'tyranny by its people who were able to live peacefully without fear of being '
 'attacked at will because they did so freely; but what about those Russians '
 'whose lives became more difficult due their own actions against us during '
 "World War II when there wasn't much resistance among them even though many "
 'others fought back after having suffered some kind attack upon themselves '
 'while fighting alongside you guys (and also your troops)? And how do these '
 'things affect American citizens today if no one can see through this '
 'propaganda machine desig

### Compare ru llama from gusev and our

### dataset IlyaGusev/ru_turbo_alpaca

In [59]:
from datasets import load_dataset

dataset = load_dataset("IlyaGusev/ru_turbo_alpaca")
dataset = dataset["train"].filter(lambda x: x["label"] == "ok")
dataset = dataset.train_test_split(test_size=500, seed=42)

No config specified, defaulting to: ru_turbo_alpaca/default
Found cached dataset ru_turbo_alpaca (/home/kosenko/.cache/huggingface/datasets/IlyaGusev___ru_turbo_alpaca/default/0.0.1/a2a1f5b065b9e34022f6bc402785c2f5fa791930917ce4f1b8d4e634def7496d)
100%|██████████| 1/1 [00:00<00:00,  2.29it/s]
Loading cached processed dataset at /home/kosenko/.cache/huggingface/datasets/IlyaGusev___ru_turbo_alpaca/default/0.0.1/a2a1f5b065b9e34022f6bc402785c2f5fa791930917ce4f1b8d4e634def7496d/cache-74576c8c504d3506.arrow
Loading cached split indices for dataset at /home/kosenko/.cache/huggingface/datasets/IlyaGusev___ru_turbo_alpaca/default/0.0.1/a2a1f5b065b9e34022f6bc402785c2f5fa791930917ce4f1b8d4e634def7496d/cache-4e79bf5937ac7374.arrow and /home/kosenko/.cache/huggingface/datasets/IlyaGusev___ru_turbo_alpaca/default/0.0.1/a2a1f5b065b9e34022f6bc402785c2f5fa791930917ce4f1b8d4e634def7496d/cache-51a1e885106bb22a.arrow


In [60]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 2405
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 500
    })
})

In [61]:
for i, item in enumerate(dataset["test"]):
    instruction = item["instruction"]
    input_string = item["input"]
    output = item["output"]
    print(instruction)
    print(input_string)
    print(output)
    print("-" * 100)
    if i > 20:
        break

Напишите краткое эссе на тему "История развития компьютерной техники".

История развития компьютерной техники началась в середине XX века, когда был разработан первый электронный компьютер. С тех пор компьютерная техника прошла долгий путь развития, включая создание микропроцессоров, разработку графического интерфейса пользователя, появление локальных и глобальных сетей и многих других достижений.
----------------------------------------------------------------------------------------------------
Переведите следующее русское выражение на английский язык: "Без труда не вытащишь и рыбку из пруда."

"No pain, no gain."
----------------------------------------------------------------------------------------------------
Определи термин: "инфляция"

Инфляция - это общее увеличение уровня цен на товары и услуги в экономике страны. Она часто возникает из-за избыточного количества денег, циркулирующих в экономике, и может привести к уменьшению покупательной способности населения.
--------------

: 

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "IlyaGusev/llama_7b_ru_turbo_alpaca_lora"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,  # load_in_8bit=True, device_map={"": 1}
)
model = PeftModel.from_pretrained(model, MODEL_NAME)
model.to("cuda:1")
model.eval()
None

In [ ]:
inputs = [
    'Что тут не так по смыслу: "зеленые бесцветные идеи яростно спят"?',
    "Brainstorm ideas for how to use a bottle of ink.",
    "Вопрос: Почему трава зеленая?\n\nВыход:",
    "Задание: Сочини длинный рассказ, обязательно упоминая следующие объекты.\nВход: Таня, мяч\nВыход:",
    "Могут ли в природе встретиться в одном месте белый медведь и пингвин? Если нет, то почему?\n\n",
    "Задание: Заполни пропуски в предложении.\nВход: Я пытался ____ от маньяка, но он меня настиг\nВыход:",
    "Как приготовить лазанью?\n\n",
    "Реши уравнение 4x + 5 = 21",
]

from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

for inp in inputs:
    data = tokenizer([inp], return_tensors="pt")
    data = {
        k: v.to(model.device)
        for k, v in data.items()
        if k in ("input_ids", "attention_mask")
    }
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    print(tokenizer.decode(output_ids, skip_special_tokens=True))
    print()
    print("==============================")
    print()

In [1]:
from optimum.bettertransformer import BetterTransformer
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "facebook/xglm-7.5B",  # torch_dtype=torch.float16
)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
for name, param in model.named_parameters():
    for num in [31]:
        if not str(num) in str(name):
            param.requires_grad = False

In [ ]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

In [2]:
model

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256008, 4096, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-31): 32 x XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=4096, out_features=16384, bias=True)
        (fc2): Linear(in_features=16384, out_features=4096, bias=True)
        (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affi